In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
model = keras.models.load_model('xception_v4_large_08_0.894.h5')

In [9]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [8]:
img = load_img('pants.jpg', target_size=(299,299))

In [11]:
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [12]:
X.shape

(1, 299, 299, 3)

In [14]:
preds = model.predict(X)

1/1 [==============================] - 0s 82ms/step


In [15]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [16]:
dict(zip(classes, preds[0]))

{'dress': -1.868289,
 'hat': -4.7612453,
 'longsleeve': -2.3169827,
 'outwear': -1.0625683,
 'pants': 9.887156,
 'shirt': -2.8124328,
 'shoes': -3.6662822,
 'shorts': 3.2003593,
 'skirt': -2.602338,
 't-shirt': -4.835047}

### convert keras to TF-lite

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2023-09-20 12:28:58.894677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-09-20 12:29:00.021623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpccdgzx9z/assets


INFO:tensorflow:Assets written to: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpccdgzx9z/assets
2023-09-20 12:29:08.417128: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-20 12:29:08.417147: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-20 12:29:08.417890: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpccdgzx9z
2023-09-20 12:29:08.435370: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-09-20 12:29:08.435390: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/53/zdvtpqvs6qxf5ht0lhmgw37c0000gp/T/tmpccdgzx9z
2023-09-20 12:29:08.493453: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-09-20 12:29:08.508766: I tensorflow/cc/saved_model/load

In [19]:
import tensorflow.lite as tflite

In [21]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
VERBOSE: Replacing 104 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact output tensor values may have changed. If such output values are checked in your code, like in your tests etc., please consider increasing error tolerance for the check.


In [24]:
input_index = interpreter.get_input_details()[0]['index']

In [27]:
output_index = interpreter.get_output_details()[0]['index']

In [28]:
interpreter.set_tensor(input_index, X)

In [29]:
interpreter.invoke()

In [31]:
preds = interpreter.get_tensor(output_index)

In [32]:
dict(zip(classes, preds[0]))

{'dress': -1.8682926,
 'hat': -4.7612476,
 'longsleeve': -2.316982,
 'outwear': -1.0625714,
 'pants': 9.887159,
 'shirt': -2.8124309,
 'shoes': -3.6662874,
 'shorts': 3.200365,
 'skirt': -2.6023397,
 't-shirt': -4.8350477}

In [33]:
from PIL import Image

In [34]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [35]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [36]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [37]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [38]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8682926,
 'hat': -4.7612476,
 'longsleeve': -2.316982,
 'outwear': -1.0625714,
 'pants': 9.887159,
 'shirt': -2.8124309,
 'shoes': -3.6662874,
 'shorts': 3.200365,
 'skirt': -2.6023397,
 't-shirt': -4.8350477}